In [1]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode
import spacy
import nltk
import stanza
import torch
from transformers import AutoTokenizer
from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
from huggingface_hub import login
from collections import Counter
from itertools import islice
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Lectura y análisis de n-gramas del archivo PDF (BobDylan.pdf)
# ---------------------------------------------------------------
# Este script realiza, paso a paso:
# 1) Lectura del PDF (ubicado en /mnt/data/BobDylan.pdf)
# 2) Preprocesamiento básico (limpieza y tokenización simple en español)
# 3) Extracción y conteo de n-gramas (unigramas, bigramas, trigramas de palabras
#    y 3-gramas de caracteres)
# 4) Visualizaciones (gráficas separadas y guardadas como PNG)
# 5) Modelo simple de bigramas con suavizado Laplace para predecir la siguiente palabra
#
# Notas:
# - Las gráficas usan matplotlib (sin estilos o colores explícitos).
# - Intento usar PyPDF2 para extraer texto; si no está disponible, pruebo con fitz (PyMuPDF).
# - Cuando sea útil, intento mostrar DataFrames con caas_jupyter_tools.display_dataframe_to_user.
# - Los archivos resultantes se guardan en /mnt/data para que puedas descargarlos.
#
# Ejecuta todo el bloque y revisa las salidas y las imágenes guardadas.

# --- 1) Lectura del PDF ---
from pathlib import Path
pdf_path = Path("C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_4/BobDylan.pdf")
assert pdf_path.exists(), f"Archivo no encontrado: {pdf_path}"

text = ""
try:
    # Primer intento: PyPDF2
    from PyPDF2 import PdfReader
    reader = PdfReader(str(pdf_path))
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
except Exception as e_py:
    try:
        # Fallback: PyMuPDF (fitz)
        import fitz
        doc = fitz.open(str(pdf_path))
        for page in doc:
            text += page.get_text() + "\n"
    except Exception as e_fitz:
        raise RuntimeError("No fue posible extraer texto del PDF con PyPDF2 ni PyMuPDF.") from e_fitz

print("Longitud del texto extraído (caracteres):", len(text))
print("Primeros 600 caracteres del texto:\n")
print(text[:1000])

Longitud del texto extraído (caracteres): 5026
Primeros 600 caracteres del texto:

Discurso del Banquete Nobel, 10 de diciembre de 2016  
Buenas noches a todos. Extiendo mis más cálidos saludos a los miembros de la Academia 
Sueca y a todos los demás distinguidos invitados que asisten esta noche.  
Lamento no poder estar con ustedes en persona, pero por favor sepan que estoy sin duda con 
ustedes en espíritu y me siento honrado de recibir un premio tan prestigioso. Ser galardonado 
con el Premio Nobel de Literatura es algo que jamás podría haber imagin ado o previsto. 
Desde una edad temprana, he estado familiarizado con, leyendo y absorbiendo las obras de 
aquellos que fueron considerados dignos de tal distinción: Kipling, Shaw, Thomas Mann, 
Pearl Buck, Albert Camus, Hemingway. Estos gigantes de la litera tura, cuyas obras se 
enseñan en las aulas, se guardan en bibliotecas de todo el mundo y se mencionan en tonos 
reverentes, siempre han dejado en mí una profunda impresión. Que ahor

In [4]:
# --- 2) Preprocesamiento y tokenización simple en español ---
import re
def clean_whitespace(s):
    return re.sub(r"\s+", " ", s).strip()

raw = clean_whitespace(text)

# Tokenizador simple (palabras) para español: conserva letras con tilde y ñ
def tokenize_words_spanish(s):
    s = s.lower()
    # Considera letras a-z y vocales acentuadas + ñ, y permite contracciones simples
    tokens = re.findall(r"[a-záéíóúüñ]+(?:'[a-záéíóúüñ]+)?", s)
    return tokens

words = tokenize_words_spanish(raw)
print("\nNúmero total de tokens (palabras):", len(words))
print("Ejemplo (primeros 40 tokens):", words[:40])

# Frases (sentencias) sencillas usando punto como separador (para algunas visualizaciones)
sentences = [clean_whitespace(s) for s in re.split(r"[.\n\r]+", raw) if s.strip()]
print("Número aproximado de oraciones detectadas:", len(sentences))



Número total de tokens (palabras): 817
Ejemplo (primeros 40 tokens): ['discurso', 'del', 'banquete', 'nobel', 'de', 'diciembre', 'de', 'buenas', 'noches', 'a', 'todos', 'extiendo', 'mis', 'más', 'cálidos', 'saludos', 'a', 'los', 'miembros', 'de', 'la', 'academia', 'sueca', 'y', 'a', 'todos', 'los', 'demás', 'distinguidos', 'invitados', 'que', 'asisten', 'esta', 'noche', 'lamento', 'no', 'poder', 'estar', 'con', 'ustedes']
Número aproximado de oraciones detectadas: 44


In [5]:
# --- 3) Funciones de n-gramas y conteo ---
from collections import Counter

def word_ngrams(tokens, n):
    if len(tokens) < n:
        return []
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def char_ngrams(s, n, pad=True):
    s2 = re.sub(r"\s+", " ", s)
    if pad:
        s2 = "_" + s2 + "_"
    return [s2[i:i+n] for i in range(len(s2)-n+1)]

In [6]:
# Conteos
unigrams = Counter(words)
bigrams = Counter(word_ngrams(words, 2))
trigrams = Counter(word_ngrams(words, 3))
char3 = Counter(char_ngrams(raw.lower(), 3))

# DataFrames de los top-n para visualización
import pandas as pd
TOP = 50

df_uni = pd.DataFrame(unigrams.most_common(TOP), columns=["token", "count"])
df_bi = pd.DataFrame([(" ".join(k), v) for k, v in bigrams.most_common(TOP)], columns=["ngram", "count"])
df_tri = pd.DataFrame([(" ".join(k), v) for k, v in trigrams.most_common(TOP)], columns=["ngram", "count"])
df_char3 = pd.DataFrame(char3.most_common(TOP), columns=["ngram", "count"])

# Mostrar tablas si caas_jupyter_tools está disponible
try:
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user("Top unigrama (palabras)", df_uni)
    display_dataframe_to_user("Top bigrama (palabras)", df_bi)
    display_dataframe_to_user("Top trigrama (palabras)", df_tri)
    display_dataframe_to_user("Top 3-gramas (caracteres)", df_char3)
except Exception:
    # Fallback: print parcial
    print("\nTop 10 Unigramas:\n", df_uni.head(10).to_string(index=False))
    print("\nTop 10 Bigramas:\n", df_bi.head(10).to_string(index=False))
    print("\nTop 10 Trigramas:\n", df_tri.head(10).to_string(index=False))
    print("\nTop 10 3-gramas (caracteres):\n", df_char3.head(10).to_string(index=False))


Top 10 Unigramas:
 token  count
   de     35
  que     33
   en     31
    y     18
   la     16
   el     16
    a     14
 para     12
  mis      9
  más      9

Top 10 Bigramas:
         ngram  count
        en la      6
        de la      4
        en el      4
 premio nobel      3
     el mundo      3
       de que      3
mis canciones      3
       lo que      3
     nobel de      2
      a todos      2

Top 10 Trigramas:
                ngram  count
   la academia sueca      2
      con ustedes en      2
     el premio nobel      2
       todo el mundo      2
       de que estaba      2
     quiénes son los      2
para estas canciones      2
         en la radio      2
     haciendo lo que      2
   son mis canciones      2

Top 10 3-gramas (caracteres):
 ngram  count
   de     48
   es     47
  do      43
  os      41
  est     39
  de      38
  que     37
  en      37
   qu     36
   en     34


In [13]:
import os
import matplotlib.pyplot as plt

# Crear el directorio si no existe
os.makedirs('/mnt/data/', exist_ok=True)

plt.rcParams["figure.dpi"] = 100
out_files = []


In [16]:
import matplotlib.pyplot as plt

# Usar un directorio que seguro existe
output_dir = "./output/"  # Directorio actual
# o
output_dir = "/tmp/"      # Directorio temporal

plt.rcParams["figure.dpi"] = 100
out_files = []

# Modificar las rutas en tu función:
plot_horizontal_bar(df_uni, "token", "count", "Top {} Unigramas (palabras)".format(TOP), 
                   f"{output_dir}top_unigramas.png", top=30, figsize=(8,10))

NameError: name 'plot_horizontal_bar' is not defined

In [15]:
# --- 5) Modelo simple de bigramas con suavizado Laplace (autocompletado) ---
V = len(unigrams)
alpha = 1.0  # Laplace
def prob_bigram(prev, w):
    # prev and w are strings (tokens)
    return (bigrams.get((prev, w), 0) + alpha) / (unigrams.get(prev, 0) + alpha * V)

def predict_next(prev, k=10):
    # devuelve los k tokens más probables tras 'prev' con probabilidad estimada
    candidates = [(w, prob_bigram(prev, w)) for w in unigrams.keys()]
    candidates_sorted = sorted(candidates, key=lambda x: x[1], reverse=True)
    return candidates_sorted[:k]

# Ejemplos de predicción usando palabras frecuentes en el texto
ejemplos_prev = ["el", "la", "y", "que", "dylan", "bob", "mi"]
print("\nEjemplos de predicción (bigramas con Laplace):")
for p in ejemplos_prev:
    preds = predict_next(p, k=8)
    # filtrar predicciones no informativas (por ejemplo tokens raros)
    print(f"\nContexto previo: '{p}' -> top predicciones:")
    for tok, prob in preds[:8]:
        print(f"   {tok}  (p≈{prob:.4f})")

# Guardar una pequeña tabla con ejemplos de predicción
preds_table = []
for p in ejemplos_prev:
    for tok, prob in predict_next(p, k=5):
        preds_table.append((p, tok, prob))
df_preds = pd.DataFrame(preds_table, columns=["prev", "next", "prob"])
try:
    display_dataframe_to_user("Predicciones Bigram (ejemplos)", df_preds)
except Exception:
    print("\nPredicciones (ejemplos):\n", df_preds.head(20).to_string(index=False))

# --- Fin del análisis ---
print("\nAnálisis completado. Si quieres, puedo:")
print(" - Explicar los gráficos uno por uno en detalle.")
print(" - Guardar un CSV con las tablas de n-gramas.")
print(" - Construir un modelo más sofisticado (Kneser-Ney, backoff) o entrenar en segmentos del texto.")


Ejemplos de predicción (bigramas con Laplace):

Contexto previo: 'el' -> top predicciones:
   mundo  (p≈0.0092)
   premio  (p≈0.0069)
   honor  (p≈0.0046)
   hecho  (p≈0.0046)
   año  (p≈0.0046)
   gran  (p≈0.0046)
   escenario  (p≈0.0046)
   financiamiento  (p≈0.0046)

Contexto previo: 'la' -> top predicciones:
   academia  (p≈0.0069)
   radio  (p≈0.0069)
   vida  (p≈0.0069)
   más  (p≈0.0046)
   litera  (p≈0.0046)
   luna  (p≈0.0046)
   gran  (p≈0.0046)
   idea  (p≈0.0046)

Contexto previo: 'y' -> top predicciones:
   me  (p≈0.0069)
   a  (p≈0.0046)
   que  (p≈0.0046)
   en  (p≈0.0046)
   estoy  (p≈0.0046)
   absorbiendo  (p≈0.0046)
   se  (p≈0.0046)
   mujeres  (p≈0.0046)

Contexto previo: 'que' -> top predicciones:
   estoy  (p≈0.0067)
   estaba  (p≈0.0067)
   de  (p≈0.0044)
   asisten  (p≈0.0044)
   me  (p≈0.0044)
   el  (p≈0.0044)
   es  (p≈0.0044)
   jamás  (p≈0.0044)

Contexto previo: 'dylan' -> top predicciones:
   discurso  (p≈0.0024)
   del  (p≈0.0024)
   banquete  (p≈0.002

In [7]:
out_files_extra = []

In [ ]:
# 1) Nube de palabras (unigramas)
wc = WordCloud(width=800, height=500, background_color="white",
               colormap="viridis").generate_from_frequencies(unigrams)

fig, ax = plt.subplots(figsize=(10,6))
ax.imshow(wc, interpolation="bilinear")
ax.axis("off")
ax.set_title("Nube de palabras (Unigramas más frecuentes)")
plt.tight_layout()
fname = "/mnt/data/wordcloud_unigramas.png"
fig.savefig(fname)
plt.show()
out_files_extra.append(fname)